In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.

import torch
from torch import Tensor, jit
from torch.nn import functional as F
from torch.optim import SGD
from torch.linalg import matrix_norm

from linodenet.models.encoders.invertible_layers import LinearContraction
from linodenet.lib import singular_triplet

torch.autograd.set_detect_anomaly(True)

In [ ]:
N, m, n = 32, 5, 3

x = torch.randn(m)
X = torch.randn(N, m)
model = jit.script(LinearContraction(m, n))
jit.save(model, "model.pt")
model = jit.load("model.pt")
optim = SGD(model.parameters(), lr=0.5)
# print(model.weight)
# print(model.cached_weight)
# model.reset_cache()
# print(model.cached_weight)
print(model.cached_weight)
model.reset_cache()
model.cached_weight

In [ ]:
print(model.weight)
print(model.cached_weight)
with torch.no_grad():
    model.weight.copy_(model.cached_weight)
print(model.weight)
print(model.cached_weight)

In [ ]:
model.zero_grad(set_to_none=True)

model.reset_cache()
model.projection()
print(model.sigma)
print(matrix_norm(model.weight, ord=2))
print(matrix_norm(model.cached_weight, ord=2))

y = -F.linear(x, model.cached_weight).norm()
y.backward()
optim.step()
print(y)
print(model.sigma)
print(matrix_norm(model.weight, ord=2))
print(matrix_norm(model.cached_weight, ord=2))

In [ ]:
import pytest

In [ ]:
?pytest.mark.flaky

In [ ]:
torch.tensor(float("inf")).any()

In [ ]:
torch.zeros(5).all()

In [ ]:
torch.randn(5).any()

In [ ]:
dir(model.cached_weight)

In [ ]:
model.zero_grad(set_to_none=True)
y = F.linear(x, model.cached_weight).norm()
y.backward()
optim.step()
model.reset_cache()
model.cached_weight